In [57]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models

# This is for the progress bar.
from tqdm import tqdm
# import seaborn as sns

In [58]:
train_df = pd.read_csv('./data/classify-leaves/train.csv')
train_df.head(10)


,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera
5,images/5.jpg,maclura_pomifera
6,images/6.jpg,ulmus_rubra
7,images/7.jpg,broussonettia_papyrifera
8,images/8.jpg,maclura_pomifera
9,images/9.jpg,broussonettia_papyrifera


In [59]:
# 各类别出现次数
train_df['label'].value_counts()

label
maclura_pomifera            353
ulmus_rubra                 235
prunus_virginiana           223
acer_rubrum                 217
broussonettia_papyrifera    214
                           ... 
cedrus_deodara               58
ailanthus_altissima          58
crataegus_crus-galli         54
evodia_daniellii             53
juniperus_virginiana         51
Name: count, Length: 176, dtype: int64

In [60]:
class_labels = train_df['label'].unique()
num_classes = len(class_labels)
num_classes

176

In [61]:
# 类名转idx
class_to_idx = {label: i for i, label in enumerate(class_labels)}
class_to_idx


{'maclura_pomifera': 0,
 'ulmus_rubra': 1,
 'broussonettia_papyrifera': 2,
 'prunus_virginiana': 3,
 'acer_rubrum': 4,
 'cryptomeria_japonica': 5,
 'staphylea_trifolia': 6,
 'asimina_triloba': 7,
 'diospyros_virginiana': 8,
 'tilia_cordata': 9,
 'ulmus_pumila': 10,
 'quercus_muehlenbergii': 11,
 'juglans_cinerea': 12,
 'cercis_canadensis': 13,
 'ptelea_trifoliata': 14,
 'acer_palmatum': 15,
 'catalpa_speciosa': 16,
 'abies_concolor': 17,
 'eucommia_ulmoides': 18,
 'quercus_montana': 19,
 'koelreuteria_paniculata': 20,
 'liriodendron_tulipifera': 21,
 'styrax_japonica': 22,
 'malus_pumila': 23,
 'prunus_sargentii': 24,
 'cornus_mas': 25,
 'magnolia_virginiana': 26,
 'ostrya_virginiana': 27,
 'magnolia_acuminata': 28,
 'ilex_opaca': 29,
 'acer_negundo': 30,
 'fraxinus_nigra': 31,
 'pyrus_calleryana': 32,
 'picea_abies': 33,
 'chionanthus_virginicus': 34,
 'carpinus_caroliniana': 35,
 'zelkova_serrata': 36,
 'aesculus_pavi': 37,
 'taxodium_distichum': 38,
 'carya_tomentosa': 39,
 'picea_p

In [62]:
# idx转类名
idx_to_class = {v : k for k, v in class_to_idx.items()}
idx_to_class

{0: 'maclura_pomifera',
 1: 'ulmus_rubra',
 2: 'broussonettia_papyrifera',
 3: 'prunus_virginiana',
 4: 'acer_rubrum',
 5: 'cryptomeria_japonica',
 6: 'staphylea_trifolia',
 7: 'asimina_triloba',
 8: 'diospyros_virginiana',
 9: 'tilia_cordata',
 10: 'ulmus_pumila',
 11: 'quercus_muehlenbergii',
 12: 'juglans_cinerea',
 13: 'cercis_canadensis',
 14: 'ptelea_trifoliata',
 15: 'acer_palmatum',
 16: 'catalpa_speciosa',
 17: 'abies_concolor',
 18: 'eucommia_ulmoides',
 19: 'quercus_montana',
 20: 'koelreuteria_paniculata',
 21: 'liriodendron_tulipifera',
 22: 'styrax_japonica',
 23: 'malus_pumila',
 24: 'prunus_sargentii',
 25: 'cornus_mas',
 26: 'magnolia_virginiana',
 27: 'ostrya_virginiana',
 28: 'magnolia_acuminata',
 29: 'ilex_opaca',
 30: 'acer_negundo',
 31: 'fraxinus_nigra',
 32: 'pyrus_calleryana',
 33: 'picea_abies',
 34: 'chionanthus_virginicus',
 35: 'carpinus_caroliniana',
 36: 'zelkova_serrata',
 37: 'aesculus_pavi',
 38: 'taxodium_distichum',
 39: 'carya_tomentosa',
 40: 'pic

In [63]:
# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # 转成pytorch tensor
    transforms.ToTensor(),
    # 随机水平反转
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_file_path(file):
    return os.path.join('data', 'classify-leaves', file)

# 定义数据集
class ImageDataset(Dataset):
    def __init__(self, csv_file, test_mode=False, transform=None):
        self.data = pd.read_csv(csv_file)
        self.test_mode = test_mode
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = get_file_path(self.data.iloc[idx, 0])
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)

        if self.test_mode:
            # 测试数据集没有label, 
            return image
        else:
            label = self.data.iloc[idx, 1]
            label_idx = class_to_idx[label]
            return image, label_idx


In [64]:
# 加载CSV文件并创建数据集  
all_dataset = ImageDataset(get_file_path('train.csv'), transform=transform)

# 划分数据集为训练集和验证集  
total_len = len(all_dataset)  
val_len = int(0.2 * total_len)  
train_len = total_len - val_len  
train_dataset, val_dataset = random_split(all_dataset, [train_len, val_len])

# 创建数据加载器  
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

In [65]:
# cpu还是gpu
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [66]:
# 定义模型，加载预训练的weight
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
# 获取模型最后一个全连接层（通常被称为fc层或分类层）的输入特征数量
num_ftrs = model.fc.in_features
# 替换全连接层
model.fc = nn.Linear(num_ftrs, len(class_to_idx))

In [67]:
# 超参数
lr = 0.0001
weight_decay = 0.001
num_epochs = 20
model_save = './classify-leaves.pth'

In [68]:
model = model.to(device) 
model.device = device

# loss.
criterion = nn.CrossEntropyLoss()

# 优化函数.
optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay=weight_decay)

In [69]:
# 训练
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # 设置模型为训练模式
        running_loss = 0.0

        for (inputs, labels) in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 累计损失
            running_loss += loss.item()

        # 在验证集上评估模型
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
          
        # 打印训练集和验证集的结果
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}, Val Acc: {val_acc:.4f}')
          
        # 保存最佳模型
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), model_save)
            print(f'Model improved, new best accuracy is {best_acc:.4f}, saving model to {model_save}')


# 评估
def evaluate_model(model, data_loader, criterion):
    # 设置模型为评估模式
    model.eval()

    # 累加损失值
    total_loss = 0.0
    # 预测正确的样本数
    correct_num = 0
    # 验证集中总的样本数
    inputs_num = 0

    with torch.no_grad():
        for (inputs, labels) in tqdm(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            # 前向传播
            outputs = model(inputs)
            # 计算损失并累加
            batch_loss = criterion(outputs, labels)
            total_loss += batch_loss.item()

            # 沿着维度1（即每个样本的输出向量）进行操作，返回两个张量：
            # 一个包含每个输出向量中的最大值，另一个包含这些最大值的索引
            _, predicted = torch.max(outputs, 1)

            # 累加batch中的样本数
            inputs_num += inputs.size(0)
            # 正确的预测数
            correct_num += (predicted == labels).sum().item()
  
    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct_num / inputs_num
    return avg_loss, accuracy

In [70]:
# 开始训练
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=num_epochs)

100%|██████████| 115/115 [00:02<00:00, 45.16it/s]


Epoch 1/20, Loss: 2.6609945162189295, Val Loss: 1.2696688859359078, Val Acc: 73.4877
Model improved, new best accuracy is 73.4877, saving model to ./classify-leaves.pth


100%|██████████| 115/115 [00:02<00:00, 44.80it/s]


Epoch 2/20, Loss: 0.8692675817895819, Val Loss: 0.5809299088042715, Val Acc: 86.0763
Model improved, new best accuracy is 86.0763, saving model to ./classify-leaves.pth


100%|██████████| 115/115 [00:02<00:00, 44.80it/s]


...

Epoch 18/20, Loss: 0.0782402461670615, Val Loss: 0.3445411283036937, Val Acc: 90.5722


100%|██████████| 115/115 [00:02<00:00, 43.99it/s]


Epoch 19/20, Loss: 0.10759043782007979, Val Loss: 0.2983164889656979, Val Acc: 92.0981


100%|██████████| 115/115 [00:02<00:00, 44.54it/s]

Epoch 20/20, Loss: 0.08591710335183442, Val Loss: 0.2541945234912893, Val Acc: 92.8338


In [72]:
# 加载最佳模型
model.load_state_dict(torch.load(model_save))
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [73]:
test_dataset = ImageDataset(get_file_path('test.csv'), test_mode=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

pred_idx = []
# Iterate the testing set by batches.
for inputs in tqdm(test_loader):
    inputs = inputs.to(device)
    with torch.no_grad():
        output = model(inputs)
        pred_idx.extend(output.argmax(dim=1).tolist())


pred_label = [idx_to_class[idx] for idx in pred_idx]


100%|██████████| 275/275 [00:05<00:00, 46.63it/s]


In [74]:
submit_file = os.path.join('.', 'submission', 'classify-leaves.csv')

test_data = pd.read_csv(get_file_path('test.csv'))
test_data['label'] = pd.Series(pred_label)
submission = pd.concat([test_data['image'], test_data['label']], axis=1)
submission.to_csv(submit_file, index=False)